In [ ]:
from torch import nn
import librosa
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torchaudio

In [ ]:
import numpy as np

In [ ]:
data_path = "TrainData"
train_x,train_y,test_x,test_y,valid_x,valid_y = [],[],[],[],[],[]
data_rate = 80

In [ ]:
cd /content/drive/MyDrive/KsponSpeech_data/10.한국어음성/KsponSpeech_data_unzip_data

In [ ]:
cd KsponSpeech_01

In [ ]:
KsponSpeech_01_path = "/content/drive/MyDrive/KsponSpeech_data/10.한국어음성/KsponSpeech_data_unzip_data/KsponSpeech_01"

In [ ]:
import scipy.io.wavfile as wav

def load_pcm(file_path, sample_rate=16000):
    # PCM 파일을 바이너리 모드로 읽기
    with open(file_path, 'rb') as pcm_file:
        pcm_data = pcm_file.read()

    # 바이트 데이터를 정수 배열로 변환 (일반적으로 16비트 PCM 가정)
    audio = np.frombuffer(pcm_data, dtype=np.int16)

    # 필요한 경우 float32로 정규화 (-1 ~ 1 범위)
    audio = audio.astype(np.float32) / 32768.0

    return audio, sample_rate

In [ ]:
def load_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read().strip()
    return text

In [ ]:
import glob
import os
base_directory = "/content/drive/MyDrive/KsponSpeech_data/10.한국어음성/KsponSpeech_data_unzip_data/KsponSpeech_01/KsponSpeech_0001/"

In [ ]:
import os

def get_filenames_files(directory):
    # 해당 디렉토리의 모든 파일 이름만 가져옴 (경로 제외)
    filenames = []

    for full_filename in os.listdir(directory):
        # full_filename은 이미 파일 이름만 포함하고 있음 (경로 없음)
        file_path = os.path.join(directory, full_filename)

        # 파일인 경우에만 리스트에 추가 (폴더는 제외)
        if os.path.isfile(file_path):
            filenames.append(full_filename)

    return filenames

# 사용 예시
files = get_filenames_files(base_directory)

In [ ]:
count = 0
for text in files:
  if count == 500 : break
  else:
    temp = text.split(".")
    if temp[1] == "pcm":
      train_x.append(text)
      train_y.append(temp[0]+".txt")
print(train_x)
print(train_y)

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def read_and_quantize_pcm(file_path, sr=16000, bit_depth=16, num_levels=16):
    """
    PCM 파일을 읽고 지정된 레벨로 양자화합니다.
    """
    # PCM 파일 읽기 (16비트 부호 있는 정수 가정)
    if bit_depth == 16:
        raw_data = np.fromfile(file_path, dtype=np.int16)
    elif bit_depth == 8:
        raw_data = np.fromfile(file_path, dtype=np.int8)
    else:
        raw_data = np.fromfile(file_path, dtype=np.int16)  # 기본값

    # [-1, 1] 범위로 정규화
    max_value = float(2 ** (bit_depth - 1))
    normalized_data = raw_data.astype(np.float32) / max_value

    # 지정된 레벨로 양자화
    min_val, max_val = -1.0, 1.0
    step = (max_val - min_val) / (num_levels - 1)
    levels = np.linspace(min_val, max_val, num_levels)

    # 가장 가까운 양자화 레벨로 매핑
    quantized_data = np.zeros_like(normalized_data)
    for i, level in enumerate(levels):
        if i == 0:
            mask = (normalized_data <= (level + step/2))
        elif i == len(levels) - 1:
            mask = (normalized_data > (level - step/2))
        else:
            mask = (normalized_data > (level - step/2)) & (normalized_data <= (level + step/2))

        quantized_data[mask] = level

    return quantized_data

# PCM 파일 경로
pcm_file = base_directory + train_x[0]  # 여기에 실제 파일 경로를 입력하세요

# PCM 파일 읽고 양자화 (예: 16 레벨로 양자화)
quantized_audio = read_and_quantize_pcm(pcm_file, num_levels=16)

# librosa만으로 시각화
plt.figure(figsize=(12, 4))
librosa.display.waveshow(quantized_audio, sr=16000)
plt.title('양자화된 PCM 오디오')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa.display

# 1. 다양한 비트 깊이로 시도
# PCM 파일 읽기 (16비트 부호 있는 정수)
data_16bit = np.fromfile(pcm_file, dtype=np.int16)
# 또는 8비트
# data_8bit = np.fromfile(pcm_file, dtype=np.int8)
# 또는 32비트
# data_32bit = np.fromfile(pcm_file, dtype=np.int32)

# 정규화 (16비트 기준)
normalized_data = data_16bit.astype(np.float32) / 32768.0

# 2. 데이터 확인
print(f"데이터 길이: {len(normalized_data)}")
print(f"최소값: {normalized_data.min()}, 최대값: {normalized_data.max()}")
print(f"평균: {normalized_data.mean()}, 표준편차: {normalized_data.std()}")

# 3. 이상치(outlier) 확인 및 처리
# 히스토그램으로 데이터 분포 확인
plt.figure(figsize=(10, 4))
plt.hist(normalized_data, bins=100)
plt.title('PCM Data Distribution')
plt.tight_layout()
plt.show()

# 4. 시각화 개선 - 데이터 일부만 확인
# 처음 10000개 샘플만 보기
segment = normalized_data[:10000]  # 데이터 세그먼트

plt.figure(figsize=(12, 4))
librosa.display.waveshow(segment, sr=16000)  # 샘플레이트는 실제에 맞게 조정하세요
plt.title('PCM Audio (First 10000 Sample)')
plt.tight_layout()
plt.show()

# 5. 다른 시각화 방법 시도
plt.figure(figsize=(12, 4))
plt.plot(segment)  # matplotlib 직접 사용
plt.title('PCM Audio (Visualization by matplotlib)')
plt.tight_layout()
plt.show()

# 6. 바이트 순서(엔디안) 확인
# 리틀 엔디안과 빅 엔디안으로 모두 읽어보기
data_little = np.fromfile(pcm_file, dtype=np.dtype('<i2'))  # 리틀 엔디안
data_big = np.fromfile(pcm_file, dtype=np.dtype('>i2'))    # 빅 엔디안

# 두 방식으로 읽은 데이터 비교 시각화
plt.figure(figsize=(12, 8))
plt.subplot(2, 1, 1)
plt.plot(data_little[:10000] / 32768.0)
plt.title('Little Endian PCM')

plt.subplot(2, 1, 2)
plt.plot(data_big[:10000] / 32768.0)
plt.title('Big Endian PCM')
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def read_pcm(file_path, sr=16000, bit_depth=16):
    """
    PCM 파일을 읽어 numpy 배열로 반환합니다.
    """
    # PCM 파일 읽기 (16비트 또는 8비트 부호 있는 정수 가정)
    if bit_depth == 16:
        raw_data = np.fromfile(file_path, dtype=np.int16)
    elif bit_depth == 8:
        raw_data = np.fromfile(file_path, dtype=np.int8)
    else:
        raw_data = np.fromfile(file_path, dtype=np.int16)  # 기본값

    # [-1, 1] 범위로 정규화
    max_value = float(2 ** (bit_depth - 1))
    normalized_data = raw_data.astype(np.float32) / max_value

    return normalized_data

# PCM 파일 경로
pcm_file = base_directory + train_x[0]  # 실제 파일 경로로 변경하세요

# PCM 파일 읽기
audio_data = read_pcm(pcm_file)
sr = 16000  # 샘플링 레이트 (필요에 따라 조정)

# 파형 시각화
plt.figure(figsize=(12, 4))
librosa.display.waveshow(audio_data, sr=sr)
plt.title('PCM audio wave')
plt.tight_layout()
plt.show()

# 스펙트로그램 계산 및 시각화
D = librosa.stft(audio_data)
S_db = librosa.amplitude_to_db(np.abs(D), ref=np.max)

plt.figure(figsize=(12, 6))
librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('PCM audio spectrum')
plt.tight_layout()
plt.show()

# 멜 스펙트로그램 시각화
S = librosa.feature.melspectrogram(y=audio_data, sr=sr)
S_db_mel = librosa.power_to_db(S, ref=np.max)

plt.figure(figsize=(12, 6))
librosa.display.specshow(S_db_mel, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('PCM audio mell spectrum')
plt.tight_layout()
plt.show()

In [ ]:
import os
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Dict, List, Union

# PCM 파일 로드 함수 (기존 코드 활용)
def read_pcm(file_path, sr=16000, bit_depth=16):
    """
    PCM 파일을 읽어 numpy 배열로 반환합니다.
    """
    # PCM 파일 읽기 (16비트 또는 8비트 부호 있는 정수 가정)
    if bit_depth == 16:
        raw_data = np.fromfile(file_path, dtype=np.int16)
    elif bit_depth == 8:
        raw_data = np.fromfile(file_path, dtype=np.int8)
    else:
        raw_data = np.fromfile(file_path, dtype=np.int16)  # 기본값

    # [-1, 1] 범위로 정규화
    max_value = float(2 ** (bit_depth - 1))
    normalized_data = raw_data.astype(np.float32) / max_value

    return normalized_data, sr

# 텍스트 파일 로드 함수 (간단한 버전)
def load_text(file_path):
    # 일반적인 한국어 인코딩 시도
    encodings = ['utf-8', 'cp949', 'euc-kr']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read().strip()
            return text
        except UnicodeDecodeError:
            continue

    # 모든 인코딩 시도 실패 시
    print(f"Warning: Could not decode {file_path} with any encoding")
    return "[텍스트 로드 실패]"

# 파일 경로 가져오기 함수
def get_file_paths(base_directory):
    """
    디렉토리에서 PCM 파일과 텍스트 파일의 경로를 가져옵니다.
    """
    pcm_files = []
    txt_files = []

    for root, _, files in os.walk(base_directory):
        for file in files:
            if file.endswith('.pcm'):
                pcm_path = os.path.join(root, file)
                txt_path = os.path.join(root, file.replace('.pcm', '.txt'))

                if os.path.exists(txt_path):
                    pcm_files.append(pcm_path)
                    txt_files.append(txt_path)

    return pcm_files, txt_files

# 데이터셋 생성 함수
def create_dataset(pcm_files, txt_files, max_samples=None):
    """
    PCM 파일과 텍스트 파일로부터 데이터셋을 생성합니다.
    """
    data = {"audio": [], "text": []}

    # 최대 샘플 수 제한
    if max_samples is not None:
        pcm_files = pcm_files[:max_samples]
        txt_files = txt_files[:max_samples]

    for pcm_file, txt_file in zip(pcm_files, txt_files):
        data["audio"].append(pcm_file)
        data["text"].append(load_text(txt_file))

    return Dataset.from_dict(data)

# 데이터 전처리를 위한 collator 클래스
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 오디오 특성 추출 및 패딩
        # 여기서 features에는 이미 prepare_dataset에서 처리된 input_features와 labels가 있어야 함
        batch = self.processor.feature_extractor.pad(
            {"input_features": [feature["input_features"] for feature in features]},
            return_tensors="pt"
        )

        # 레이블(텍스트) 패딩
        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": [feature["labels"] for feature in features]},
            return_tensors="pt"
        )

        # 패딩된 부분을 -100으로 대체 (손실 계산에서 무시됨)
        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id,
            -100
        )

        batch["labels"] = labels

        return batch

# 메인 훈련 함수
def train_whisper_model():
    # 데이터 디렉토리 설정
    base_directory = "/content/drive/MyDrive/KsponSpeech_data/10.한국어음성/KsponSpeech_data_unzip_data/KsponSpeech_01"
    output_dir = "/content/drive/MyDrive/whisper_finetuned"

    # 파일 경로 가져오기
    pcm_files, txt_files = get_file_paths(base_directory)
    print(f"총 {len(pcm_files)}개의 파일이 로드되었습니다.")

    # 데이터셋 생성 (필요에 따라 샘플 수 제한)
    dataset = create_dataset(pcm_files, txt_files, max_samples=1000)

    # Audio 형식으로 변환
    def map_to_array(batch):
        arrays = []
        rates = []

        for audio_path in batch["audio"]:
            audio_array, sampling_rate = read_pcm(audio_path)
            arrays.append(audio_array)
            rates.append(sampling_rate)

        batch["audio"] = [{"array": arr, "sampling_rate": sr} for arr, sr in zip(arrays, rates)]
        return batch

    # 데이터셋에 Audio 형식 적용 (배치 처리)
    dataset = dataset.map(map_to_array, batched=True, batch_size=8)

    # 훈련/검증/테스트 세트 분할
    train_test_valid = dataset.train_test_split(test_size=0.2)
    test_valid = train_test_valid["test"].train_test_split(test_size=0.5)

    datasets = DatasetDict({
        "train": train_test_valid["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"]
    })

    # Whisper 모델 및 프로세서 로드
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")

    # 데이터 전처리를 위한 함수
    def prepare_dataset(batch):
        # 비배치(non-batched) 함수로 변경
        # 오디오 처리
        audio = batch["audio"]

        # 특성 추출
        input_features = processor.feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_features[0]

        # 텍스트 처리
        labels = processor.tokenizer(batch["text"]).input_ids

        return {
            "input_features": input_features,
            "labels": labels
        }

    # 데이터셋 전처리 적용 (비배치 함수)
    processed_datasets = DatasetDict({
        split: dataset.map(
            prepare_dataset,
            remove_columns=dataset.column_names
        )
        for split, dataset in datasets.items()
    })

    # 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    # WER(Word Error Rate) 평가 함수
    def compute_metrics(pred):
        pred_ids = pred.predictions
        label_ids = pred.label_ids

        # -100을 패딩 토큰으로 대체
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        # 예측과 레이블을 텍스트로 디코딩
        pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # WER 계산 (간단한 구현, 실제로는 더 정교한 방법 사용 가능)
        wer = sum([1 - (pred == label) for pred, label in zip(pred_str, label_str)]) / len(pred_str)

        return {"wer": wer}

    # 훈련 인자 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,
        learning_rate=1e-5,
        warmup_steps=500,
        max_steps=4000,
        gradient_checkpointing=True,
        fp16=True,
        evaluation_strategy="steps",  # 올바른 파라미터명 사용
        eval_steps=500,
        save_steps=500,
        logging_steps=25,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
    )

    # 훈련기 설정
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_datasets["train"],
        eval_dataset=processed_datasets["validation"],
        data_collator=data_collator,
        tokenizer=processor.tokenizer,  # processing_class 대신 tokenizer 사용
        compute_metrics=compute_metrics,
    )

    # 모델 훈련
    trainer.train()

    # 모델 저장
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)

    # 테스트 평가
    test_results = trainer.evaluate(processed_datasets["test"])
    print(f"테스트 결과: {test_results}")

    return model, processor

# 모델 추론 함수
def transcribe_audio(model, processor, audio_file):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    # PCM 파일 로드
    audio_array, sr = read_pcm(audio_file)

    # 특성 추출
    input_features = processor.feature_extractor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features

    # 모델을 통한 예측
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # 예측된 토큰을 텍스트로 디코딩
    transcription = processor.tokenizer.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0]

    return transcription

In [ ]:
# 메인 실행 부분
if __name__ == "__main__":
    # 모델 훈련
    model, processor = train_whisper_model()

    # 테스트 오디오 파일 변환
    test_audio = "/content/drive/MyDrive/KsponSpeech_data/10.한국어음성/KsponSpeech_data_unzip_data/KsponSpeech_01/test.pcm"
    transcription = transcribe_audio(model, processor, test_audio)
    print(f"변환 결과: {transcription}")

In [ ]:
!pip install datasets transformers librosa

librosa.load method는 audio file에서 waveform y와 sampling rate sr을 불러온다.
y는 1차원 numpy float array이다.
sr은 초당 sample의 수를 의미하며, default 값은 22050Hz이다.